1\. Write a function that converts number representation (bin<->dec<->hex)

In [6]:
def convert(x):
    if isinstance(x,str):
        return bin(int(x)),hex(int(x))
    if isinstance(x,bytes):
        try:
            bin_=int(x.decode(),2)
            return bin_,hex(bin_)
        except ValueError:
            hex_=int(x.decode(),16)
            return hex_,bin(hex_)
    else:
        raise ValueError("Wrong input!")
    
x = '255'
print(convert(x))
number2 = b'0b11111111'
print(convert(number2))
number3 = b'0xff'
print(convert(number3))

('0b11111111', '0xff')
(255, '0xff')
(255, '0b11111111')


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [163]:
import math
def convert_2(word):
    word = word.decode()
    if len(word)!=32:
        raise ValueError("Wrong input argument")
    sign = word[0]
    exponent = word[1:9]
    mantissa = word[9:]
    res = (-1)**int(sign) * float(f'1.{int(mantissa,2)}') * math.pow(2,(int(exponent,2)-127))
    return res
    
print(convert_2(b"01111111111111111111111111111111"))

6.257318714338937e+38


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [162]:
min_,max_ = 1.0,1.0
c=1
while max_!=float('inf') and c<10000:
    if max_*2!=float('inf'):
        max_*=2
    elif max_+1!=float('inf'):
        max_+=1.0
    c+=1
print(max_)
c=0
while min_!=0 and c<10000:
    if min_/2!=0:
        min_/=2
    c+=1
print(min_)
#8.98846567431158e+307

8.98846567431158e+307
5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [20]:
# (a)
import math as m
def quad_eq(a,b,c):
    x1 = (-b+m.sqrt(b**2-4*a*c))/(2*a)
    x2 = (-b-m.sqrt(b**2-4*a*c))/(2*a)
    return x1,x2
print(quad_eq(0.001,1000,0.001))

# (b)
def quad_eq2(a,b,c):
    x1 = (b**2 - (b**2 - 4*a*c))/((-2*a*b) - 2*a*m.sqrt(b**2-4*a*c))
    x2 = (b**2 - (b**2 - 4*a*c))/((-2*a*b) + 2*a*m.sqrt(b**2-4*a*c))
    return x1, x2
print(quad_eq2(0.001,1000,0.001))

# (c)
def accurate_quad_eq(a,b,c):
    x1 = (2*c)/(-b-m.sqrt(b**2 -4*a*c))
    x2 = (2*c)/(-b+m.sqrt(b**2-4*a*c))
    return x1, x2
print(accurate_quad_eq(0.001,1000,0.001))
    

(-9.999894245993346e-07, -999999.999999)
(-1.00000761449437e-06, -1000029.7368713223)
(-1.000000000001e-06, -1000010.5755125057)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
